# Pendahuluan
Modul ini akan memandu Anda dalam membangun formulir tanya jawab dengan sumber informasi PDF. Tidak seperti modul sebelumnya, modul ini tidak menggunakan LangChain sebagai orkestrator agent. Pada akhir modul ini, aplikasi akan dibungkus dalam antarmuka web yang mudah digunakan dengan Gradio.

In [ ]:
!pip -q install --upgrade openai
!pip -q install transformers accelerate datasets evaluate
!pip -q install --upgrade -i https://pypi.org/simple/ bitsandbytes
!pip -q install --upgrade langchain langchain-community pypdf chromadb
!pip -q install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [ ]:
from google.colab import userdata
from openai import OpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

LLM_Agent_Type = "Open AI" # @param ["TinyLlama", "Zephyr", "Sea Lion", "Open AI"]

def create_hf_gen(model, tokenizer):
  def hf_gen(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=2048,
    )
    gen_text = tokenizer.batch_decode(gen_tokens[:, input_ids.shape[1]:], skip_special_tokens=True)[0]
    return gen_text

  return hf_gen


llm_gen = None
match LLM_Agent_Type:

  case "Open AI":
    client = OpenAI(
        api_key=userdata.get("OAI_KEY")
    )

    def oai_gen(prompt):
      completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": prompt}
        ]
      )
      return completion.choices[0].message.content

    llm_gen = oai_gen

  case "TinyLlama":
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm_gen = create_hf_gen(model, tokenizer)

  case "Zephyr":
    model_name = "HuggingFaceH4/zephyr-7b-alpha"
    quantization_config = BitsAndBytesConfig(
      load_in_8bit=True
    )
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map=device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm_gen = create_hf_gen(model, tokenizer)

  case "Sea Lion":
    model_name = "aisingapore/sea-lion-7b-instruct"
    quantization_config = BitsAndBytesConfig(
      load_in_8bit=True
    )
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map=device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm_gen = create_hf_gen(model, tokenizer)

# Per-Step Testing

In [ ]:
from google.colab import files

# Upload PDF
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Materi Learning Telkom Design Sprint Master.pdf to Materi Learning Telkom Design Sprint Master.pdf
User uploaded file "Materi Learning Telkom Design Sprint Master.pdf" with length 8708753 bytes


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load and Split PDF file
for filename in uploaded.keys():
  loader = PyPDFLoader(filename)
  chunks = loader.load_and_split()
  # Rapikan
  for chunk in chunks:
    chunk.page_content = ' '.join(chunk.page_content.split())


print(len(chunks))

39


In [ ]:
taken_chunks = chunks[5:]
taken_chunks

[Document(page_content='MODULE 1 : DESIGN SPRINT MASTER TELKOM BNSP INTRODUCTION LEARNING OUTCOMES ● Mengetahui definisi dan peran Design Sprint Master ● Memahami proses terkait Sertifikasi Telkom Design Sprint Master BNSP Submodule 1.1 : Pengenalan Sprint Master SUBMODULE DESCRIPTION OVERVIEW Mengenali definisi terkait Sprint Master beserta tugas pokoknya dalam proses Design Sprint Sprint Master adalah seseorang yang berperan sebagai fasilitator pada proses design sprint. Dimana tugas utama dari sprint master adalah memastikan proses design sprint yang dilakukan oleh anggota tim yang dipimpin oleh sprint master tersebut berjalan sesuai dengan kaidah dan metode design sprint yang digunakan. Sprint Master biasanya dapat memimpin sampai dengan 11 anggota design sprint yang tujuan memandu, men-challenge sampai juga memberikan stimulus untuk para anggota design sprint untuk dapat menyelesaikan permasalahan yang ada. Biasanya, Perusahaan membutuhkan karyawan yang bekerja sebagai Sprint Mast

In [ ]:
# Setup Vector Database
import chromadb
from chromadb import Settings

# python can run in-memory with no server running: chromadb.PersistentClient()
client = chromadb.PersistentClient("./chromadb", settings=Settings(allow_reset=True))
client.reset() # ⚠️ not recommanded
client.list_collections()

[]

In [ ]:
# Try Embedding Function
from chromadb.utils import embedding_functions
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

doc_embedding = sentence_transformer_ef([chunks[14].page_content])
print(chunks[14].page_content)

Setelah Anda membuat prototipe, saatnya mengumpulkan feedback dari pengguna Anda. Penting bagi Anda untuk mengoptimalkan cara mengumpulkan feedback, dengan begitu Anda tidak hanya menghemat waktu dan sumber daya, tetapi juga belajar lebih banyak dari prototipe dan sesi pengujian. Ketepatan prototipe mengacu pada tingkat kelengkapan dan detailnya. Tingkat kelengkapan prototipe yang Anda buat bergantung pada tahap kemajuan prosesnya, yaitu adalah sebagai berikut; ● Low Fidelity – biaya rendah, kasar dan cepat dibuat ● Medium Fidelity – sedikit lebih detail, masih kasar tetapi lebih dekat ke solusi ● High Fidelity– lebih mendekati final, sangat detail, dan lebih memakan waktu Hal ini mewakili skala kelengkapan atau kedekatan dengan produk akhir, yang berbeda tergantung pada jenis solusi dan kebutuhan situasi. Prototipe juga dapat memiliki bagian yang berbeda dengan berbagai tingkat ketepatan. Misalnya, Anda dapat membuat prototipe dengan fidelitas visual tinggi tetapi dengan fidelitas fun

In [ ]:
print(len(doc_embedding[0]))
print(doc_embedding[0])

384
[0.011691945604979992, 0.04046075418591499, -0.04307511821389198, 0.0005346221150830388, -0.06169956922531128, -0.04689057543873787, -0.003960960078984499, 0.10208313167095184, 0.004971117712557316, -0.03096868097782135, 0.06836288422346115, -0.02495051920413971, -0.016642726957798004, -0.0808219388127327, 0.01591716706752777, -0.03383081778883934, 0.11030539870262146, 0.02044016681611538, -0.11678418517112732, -0.038957562297582626, 0.04070303961634636, -0.10201278328895569, -0.032300740480422974, -0.000674653158057481, 0.020820533856749535, -0.031541984528303146, -0.0074079278856515884, 0.016464587301015854, 0.0025365750771015882, -0.06783115118741989, -0.019849790260195732, 0.1221342608332634, 0.0013443665811792016, -0.07269074022769928, -0.024135537445545197, 0.05552861467003822, -0.020087627694010735, 0.019317995756864548, -0.015426547266542912, 0.02656579576432705, -0.02225477807223797, -0.02539759688079357, -0.06403235346078873, -0.047147054225206375, 0.04786016047000885, -0

In [ ]:
course_catalog = client.get_or_create_collection(
    name = "chunk_of_books",
    # metadata = {"hnsw:space": "cosine"},
    embedding_function = sentence_transformer_ef,
  )

course_catalog.add(
  documents = [ chunk.page_content for chunk in taken_chunks ],
  metadatas = [ chunk.metadata for chunk in taken_chunks ],
  ids = [ f"""ids-{str(hash(f"{chunk.metadata['source']} | {chunk.metadata['page']}"))}""" for chunk in taken_chunks ], # must be unique for each doc
)


In [ ]:
retrieval_results = course_catalog.query(
    query_texts=["Nilai yang dianut oleh Sprint Master"],
    n_results=1,
)
retrieval_results['documents'][0][0]

'6. Ice breaks on its own Tidak perlu membuat suatu ice breaking di luar proses design sprint, karena belum tentu semua peserta akan menerima bahkan ada yang skeptis. Biarkan berjalan apa adanya, mungkin awalnya akan terasa sedikit canggung, namun percayalah bahwa orang akan merasa nyaman. Hal itu sering terjadi bahkan mereka ada yang meminta tambahan waktu. 7. Write names on the board Penting bagi fasilitator untuk mengetahui nama - nama peserta sprint, karena percakapan akan menjadi jauh lebih intim saat Anda memanggil orang dengan namanya. Setiap kali ada orang asing, Anda dapat berkeliling ruangan dan meminta semua orang untuk memperkenalkan diri dan menulis nama mereka pada stiker yang dapat ditempelkan di bagian dada (yang dapat terlihat jelas). 8. Fake confidence (itʼs normal to feel nervous) Anda akan semakin percaya diri dari waktu ke waktu, tetapi Anda juga harus tahu bahwa merasa gugup sebelum dan selama sprint sebagai fasilitator adalah hal yang wajar. Perlu diperhatikan ba

In [ ]:
import jinja2

QNA_PROMPT_TEMPLATE = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{{ context }}

Question: {{ question }}

Answer:"""

def RAG_QnA(question):
  retrieval_results = course_catalog.query(
    query_texts=["Nilai yang dianut oleh Sprint Master"],
    n_results=1,
  )
  context = retrieval_results['documents'][0][0]
  environment = jinja2.Environment()
  template = environment.from_string(QNA_PROMPT_TEMPLATE)
  formatted_prompt = template.render(context=context, question=question)
  answer = llm_gen(formatted_prompt)
  return answer


In [ ]:
RAG_QnA("Apa saja nilai dan prinsip yang dianut oleh Telkom Sprint Master?")

'Nilai dan prinsip yang dianut oleh Telkom Sprint Master adalah menjaga proses tetap efektif, patuh terhadap batasan waktu, menghandle peserta sprint dengan baik, menghindari keaktifan atau kepasifan yang berlebihan, menjaga waktu dengan baik, memberikan konteks sebelum memulai proses sprint, dan tidak memfasilitasi lebih dari satu tim secara bersamaan kecuali jika sudah terkontrol.'

In [ ]:
llm_gen("""Question: Apa saja nilai dan prinsip yang dianut oleh Telkom Sprint Master?
Answer:""")

'1. Kepuasan Pelanggan: Telkom Sprint Master mengutamakan kepuasan pelanggan dengan memberikan pelayanan yang terbaik dan solusi yang tepat untuk memenuhi kebutuhan pelanggan.\n\n2. Integritas: Telkom Sprint Master berkomitmen untuk bertindak secara jujur, adil, dan profesional dalam semua aspek bisnisnya.\n\n3. Inovasi: Telkom Sprint Master selalu mencari cara baru untuk meningkatkan layanan dan produknya, serta terus mengikuti perkembangan teknologi terkini.\n\n4. Kerjasama Tim: Telkom Sprint Master memegang prinsip bahwa kerjasama tim yang baik akan menghasilkan kinerja terbaik dalam mencapai tujuan perusahaan.\n\n5. Keberlanjutan: Telkom Sprint Master menjunjung tinggi prinsip keberlanjutan dalam bisnisnya, dengan selalu memperhatikan dampak lingkungan dan masyarakat sekitar.'

Hal yang perlu dipertimbangkan lebih lanjut.
- Apakah embedding function cocok untuk Bahasa Indonesia?
- Ukuran _chunk_ dan _overlapping_ konten?
- Relevansi (RAGAS)?

# Wrap into Web UI

In [ ]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 44.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
def predict(question):
    answer = RAG_QnA(question)
    return answer

gr.Interface(
    predict,
    inputs=[
        gr.Textbox(lines=2, value="Tulis pertanyaanmu di sini!", label="Question"),
    ],
    outputs=[gr.Textbox(label="Answer")],
).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d5a812370a73add30f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://d5a812370a73add30f.gradio.live
